In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from utils import sparse_categorical_crossentropy_with_logits

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Softmax
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
model = tf.keras.models.load_model('model_simple', custom_objects={'foo': sparse_categorical_crossentropy_with_logits})

In [ ]:
HEIGHT = 75
WIDTH = 75

In [ ]:
labels = ['point_foreward', 'point_up']
test_label_ind = 1
# test_dir = 'images/train/{}'.format(labels[test_label_ind])
test_dir = os.path.join('frames', 'curl', )
preds = []
# test_dir = os.path.join('..', 'detect_classify_images', 'failed', 'point_foreward')
files = os.listdir(test_dir)
# print(files)
num_correct = 0
for f in files:
    img = cv2.imread(os.path.join(test_dir, f))
    img = cv2.resize(img, (HEIGHT, WIDTH))
    
    batch = np.expand_dims(img, axis=0).astype(np.float32)
    res = model.predict(batch)
    pred_ind = np.argmax(res[0])
    label = labels[pred_ind]
    preds.append(pred_ind)

    if pred_ind == test_label_ind:
        num_correct += 1
    
    print('prediction: {} | actual: {}'.format(label, labels[test_label_ind]))
#     cv2.putText(img, label, (0, 20), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,0,0), 1)
#     cv2.imshow('img', img)
#     while True:
#         k = cv2.waitKey(30) & 0xff
#         if k == ord('n'):
#             break
#         elif k == ord('q'):
#             files = []

cv2.destroyAllWindows()

print('Total correct: {} => accuracy of: {}'.format(num_correct, num_correct / len(files)))

In [ ]:
gts = np.array([0 for i in range(len(files))]).astype(np.float64)
preds = np.array(preds).astype(np.float64)
cat_acc = tf.keras.metrics.categorical_accuracy(gts, preds)
bin_acc = tf.keras.metrics.binary_accuracy(gts, preds)
spar_acc = tf.keras.metrics.sparse_categorical_accuracy(gts, preds)